Built a model which predicts the customer churn using an artificial neural network!

In [1]:
#Importing required libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

Data pre-processing

In [3]:
#Importing the data
data = pd.read_csv('Churn_Modelling.csv')
x = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [4]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


In [6]:
#Encoding categorical data

#Label encoding 'gender column'
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [7]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [12]:
#One hot encoding of geography column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [13]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [14]:
#Splitting the dataset into training and testing set

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [15]:
#Feature scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

Building the Artifical Neural Netwrok

In [16]:
#Initialising the ANN 
#Sequential groups a linear stack of layers into a tf.keras.Model
ann = tf.keras.models.Sequential()

In [17]:
#Adding the input layer and first hidden layer
# Dense is for densely-connected NN layer
#Activation for models with more classes is softmax
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [18]:
#Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [19]:
#Adding the output layer
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

Training the ANN

In [20]:
#Compiling the ANN
#For binary output loss will be binary_crossentropy and categorical_crossentropy for more classes 
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
#Training the ANN on training set
ann.fit(x_train, y_train, batch_size =32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5980 - accuracy: 0.7195
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4650 - accuracy: 0.8004
Epoch 3/100
250/250 [==============================] - 0s 979us/step - loss: 0.4368 - accuracy: 0.8054
Epoch 4/100
250/250 [==============================] - 0s 987us/step - loss: 0.4177 - accuracy: 0.8108
Epoch 5/100
250/250 [==============================] - 0s 942us/step - loss: 0.4168 - accuracy: 0.8089
Epoch 6/100
250/250 [==============================] - 0s 944us/step - loss: 0.3884 - accuracy: 0.8235
Epoch 7/100
250/250 [==============================] - 0s 988us/step - loss: 0.3943 - accuracy: 0.8143
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3777 - accuracy: 0.8207
Epoch 9/100
250/250 [==============================] - 0s 887us/step - loss: 0.3782 - accuracy: 0.8200
Epoch 10/100
250/250 [==============================] - 0s 932us/step - loss: 0

In [23]:
#Using built ANN to predict a single piece of data
print(ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

[[0.03154215]]


Predicting the test set

In [24]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the confusion matrix

In [25]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1519   76]
 [ 200  205]]


In [26]:
#Prinitng the accuracy
accuracy_score(y_test, y_pred)

0.862